In [1]:
import Parity_Data_Generator
import numpy as np
from pyESN import ESN
import time

N = 3000   # number of datapoints
n = 3       # n-parity
timescale = 50
rng = np.random.RandomState(42)



start_time = time.time()
# produce Data
bits, parity, target = Parity_Data_Generator.generateParityData(N, n, randomstate=rng)

# Divide in training and test data
traintest_cutoff = int(np.ceil(0.7 * len(bits)))
train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]

print("--- %s seconds ---" % (time.time() - start_time))

In [2]:
def runSomeConfigs(spectral_rad, leak_r, train_b, test_b, train_p, test_p):
#     print("Spectral-radius:",spectral_rad, "leak-rate:", leak_r)
    nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= leak_r,   # adjust leak_rate
                          sparsity=0.95,
                          noise=0.01,
                          input_shift=0,
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_forcing=False,
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
#                           out_activation=np.tanh,
#                           inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
    # print(np.shape(bits),np.shape(parity),np.shape(target))
    nParityESN.fit(train_b, train_p)
    pred_test = nParityESN.predict(test_b)

    #### not rounding
    predictedParity = pred_test # not rounding
    #### Rounding
#     better_pred_test = []           # Maybe don't round and return "true" results from pred_test
#     for x in pred_test:
#         better_pred_test.append(0 if x < 0.5 else 1)
#     predictedParity = np.array(better_pred_test).reshape(-1,1)
#     print(test_parity[40030:40100].reshape(-1))
#     print(predictedParity[40030:40100].reshape(-1))
    
    print("Testing error")
    test_error = np.sqrt(np.mean((predictedParity - test_p) ** 2))
    print(test_error)
    return test_error



In [ ]:
def produceTrainingData(i):
    bits, parity, target = Parity_Data_Generator.generateParityData(N, n, timescale=timescale, 
                                                                    randomstate=np.random.RandomState(i))

    # Divide in training and test data
    traintest_cutoff = int(np.ceil(0.7 * len(bits)))
    train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
    train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
#     train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]
    return train_bits, test_bits, train_parity, test_parity


In [ ]:
trials = 10
n_neurons = 250

l_rate = np.linspace(0.1, 1, num=10)
# l_rate =[0.2]
# l_rate = [0.1,0.2,0.3]

# sp_rad = np.linspace(0.9, 1.0, num=5)
sp_rad = np.linspace(0.1, 1, num=10)
# sp_rad = [0.8]

run_time = time.time()
iterations, iterationcount = trials*len(sp_rad)*len(l_rate), 0
errors = []
all_errors = [{} for _ in range(trials)] # empty dict
for i in range(trials):
    errors = []
    # produce training and testing data
    train_bits, test_bits, train_parity, test_parity = produceTrainingData(i)
    for l in l_rate:
        for s in sp_rad:
            print("Neurons:", n_neurons,"Leak-rate:",l,"Spectral-radius:",s)
            errors.append(runSomeConfigs(s,l, train_bits, test_bits, train_parity, test_parity))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors[i] = errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 250 Leak-rate: 0.1 Spectral-radius: 0.1
harvesting states...
fitting...
training error:
0.450355631637
Testing error
0.461248559769
--- Progress: 0.10% --- 0.32 minutes ---
Neurons: 250 Leak-rate: 0.1 Spectral-radius: 0.2
harvesting states...
fitting...
training error:
0.40942160509
Testing error
0.428784916176
--- Progress: 0.20% --- 0.62 minutes ---
Neurons: 250 Leak-rate: 0.1 Spectral-radius: 0.3
harvesting states...
fitting...
training error:
0.383341515652
Testing error
0.633309153187
--- Progress: 0.30% --- 0.96 minutes ---
Neurons: 250 Leak-rate: 0.1 Spectral-radius: 0.4
harvesting states...
fitting...
training error:
0.392290427222
Testing error
0.55180818714
--- Progress: 0.40% --- 1.31 minutes ---
Neurons: 250 Leak-rate: 0.1 Spectral-radius: 0.5
harvesting states...


In [3]:
import pickle
#### Save Data
with open('datasaveAverageSmallerSlowESN.p', 'wb') as outputFile:
    pickle.dump(all_errors, outputFile)
#     pickle.dump(num_neurons, outputFile)
    pickle.dump(trials, outputFile)
    pickle.dump(l_rate, outputFile)
    pickle.dump(sp_rad, outputFile)
#### Load Data
# with open('datasaveAverageSmallerSlowESN.p', 'rb') as inputFile:
#     all_errors = pickle.load(inputFile)
# #     num_neurons = pickle.load(inputFile)
#     trials = pickle.load(inputFile)
#     l_rate = pickle.load(inputFile)
#     sp_rad = pickle.load(inputFile)

In [4]:
### Average errors
errorLists = np.zeros(100)
for i in range(len(all_errors)):
    e = all_errors[i]
    errorLists = np.add(errorLists, np.array(e))

errorLists = np.divide(errorLists, trials)

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors))]
# c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]

x = np.repeat(l_rate,trials)
y = np.tile(sp_rad, trials)
z = errorLists
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name=("Averaged over", trials ,"Trials"),
    mode='markers',
    marker=dict(
        size=7,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='SlowESN average Error')

In [6]:
data = [
     go.Contour(
        z=errorLists.reshape(10,10),
        x=l_rate,
        y=sp_rad,
        colorscale='Jet',
    )]
py.iplot(data)

In [9]:
#### Get Parameters from a good performing slowESN

nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=0.5,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= 0.2,   # adjust leak_rate
                          sparsity=0.95,     # 0.95
                          noise=0.01,      # 0.001
                          input_shift=-0.5,    # 0
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_forcing=False,
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
#                           out_activation=np.tanh,
#                           inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
nParityESN.fit(train_bits, train_parity)

harvesting states...
fitting...
training error:
0.16965521365


array([[ 0.58432033],
       [ 0.78390291],
       [-0.1110263 ],
       ..., 
       [ 0.81041993],
       [ 0.92636471],
       [ 0.99699464]])

In [12]:
##### Create Training Data for fastESN with output from slowESN
start_time = time.time()

new_rng = np.random.RandomState(1337)
new_N, new_n = 1500, 3
# new_bits, new_parity, new_target = Parity_Data_Generator.generateParityData(new_N, new_n, 
#                                                                             timescale=timescale,randomstate=new_rng)

# # Divide in training and test data
# traintest_cutoff = int(np.ceil(0.7 * len(bits)))
# new_train_bits, new_test_bits = new_bits[:traintest_cutoff], new_bits[traintest_cutoff:]
# # new_train_parity, new_test_parity = new_parity[:traintest_cutoff], new_parity[traintest_cutoff:]
# new_train_targets, new_test_targets = new_target[:traintest_cutoff], new_target[traintest_cutoff:]

# train_predicted_parity = nParityESN.predict(new_train_bits)
# test_predicted_parity = nParityESN.predict(new_test_bits)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [13]:
#### FastESN ####
def runSomeFastConfigs(spectral_rad, leak_r, train_parity, test_parity, 
                       train_targets, test_targets, num_neurons=100):
    outputESN = ESN(n_inputs=1,
                              n_outputs=1,
                              n_reservoir=num_neurons,  
                              spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                              leak_rate= leak_r,   # adjust leak_rate
                              sparsity=0.95,
                              noise=0.001,
                              input_shift=0,
                              input_scaling=3,  # 3 seems to be fine. Maybe use lower
                              teacher_forcing=False,
                              teacher_scaling=1.12,
                              teacher_shift=-0.4,
#                               out_activation=np.tanh,
#                               inverse_out_activation=np.arctanh,
                              random_state=new_rng,
                              silent=False)
        # print(np.shape(bits),np.shape(parity),np.shape(target))
    outputESN.fit(train_parity, train_targets)
    pred_test = outputESN.predict(test_parity)
        
    print("Testing error")
    test_error = np.sqrt(np.mean((pred_test - test_targets) ** 2))
    print(test_error)
    return test_error

def produceFastTrainingData(randomState):
    bits, parity, target = Parity_Data_Generator.generateParityData(N, n, timescale=timescale, 
                                                                    randomstate=np.random.RandomState(randomState))

    # Divide in training and test data
    traintest_cutoff = int(np.ceil(0.7 * len(bits)))
    train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
    train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]
    
    train_predicted_parity = nParityESN.predict(train_bits)
    test_predicted_parity = nParityESN.predict(test_bits)
    
    return train_predicted_parity, test_predicted_parity, train_targets, test_targets

In [15]:
new_trials = 10
n_neurons = 100
# num_neurons_fast = [100]
# l_rate_fast = np.linspace(0.4, 1, num=7)
l_rate_fast = np.linspace(0.1, 1, num=10)
# l_rate_fast = [0.6]
# sp_rad_fast = np.linspace(0.8, 1, num=3)
sp_rad_fast = np.linspace(0.1, 1, num=10)
# sp_rad_fast = [0.95]

run_time = time.time()
iterations, iterationcount = new_trials*len(sp_rad_fast)*len(l_rate_fast), 0
errors = []
all_errors_fast = [{} for _ in range(new_trials)]
for i in range(new_trials):
    errors = []
    
    new_train_parity, new_test_parity, new_train_targets, new_test_targets = produceFastTrainingData(i)
    
    for l in l_rate_fast:
        for s in sp_rad_fast:
            print("Neurons:", n_neurons, "Leak-rate:", l, "Spectral-radius:", s)
            errors.append(runSomeFastConfigs(s,l,new_train_parity, new_test_parity, new_train_targets, new_test_targets,n_neurons))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors_fast[i] = errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.1
harvesting states...
fitting...
training error:
0.143667028733
Testing error
0.159297254471
--- Progress: 0.10% --- 0.14 minutes ---
Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.2
harvesting states...
fitting...
training error:
0.141261286032
Testing error
0.156829712898
--- Progress: 0.20% --- 0.18 minutes ---
Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.3
harvesting states...
fitting...
training error:
0.141468277116
Testing error
0.156959428714
--- Progress: 0.30% --- 0.23 minutes ---
Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.4
harvesting states...
fitting...
training error:
0.143241205838
Testing error
0.158082551254
--- Progress: 0.40% --- 0.27 minutes ---
Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.5
harvesting states...
fitting...
training error:
0.139794160331
Testing error
0.156518825651
--- Progress: 0.50% --- 0.31 minutes ---
Neurons: 100 Leak-rate: 0.1 Spectral-radius: 0.6
harvesting states...
fitting...
training 

In [15]:
import pickle
#### Save Data
with open('datasaveFastAverage.p', 'wb') as outputFile:
    pickle.dump(all_errors_fast, outputFile)
#     pickle.dump(num_neurons_fast, outputFile)
    pickle.dump(new_trials, outputFile)
    pickle.dump(l_rate_fast, outputFile)
    pickle.dump(sp_rad_fast, outputFile)
#### Load Data
# with open('datasaveFastAverage.p', 'rb') as inputFile:
#     all_errors_fast = pickle.load(inputFile)
# #      num_neurons_fast = pickle.load(inputFile)
#     new_trials = pickle.load(inputFile)
#     l_rate_fast = pickle.load(inputFile)
#     sp_rad_fast = pickle.load(inputFile)

In [16]:
### Average errors
new_trials = 10
fast_errorLists = np.zeros(100)
for i in range(len(all_errors_fast)):
    e = all_errors_fast[i]
    fast_errorLists = np.add(fast_errorLists, np.array(e))

fast_errorLists = np.divide(fast_errorLists, new_trials)

[ 0.15762748  0.15675149  0.15645941  0.15704812  0.15740527  0.15826223
  0.15790272  0.15848116  0.15783754  0.15861234  0.14795101  0.14536484
  0.14421469  0.14267869  0.14559568  0.14747176  0.14862034  0.14952055
  0.15182427  0.15109582  0.13677562  0.13223609  0.13040697  0.13223314
  0.13292161  0.13351394  0.13617894  0.13492751  0.14017693  0.14084311
  0.13030437  0.12752644  0.12365327  0.12367669  0.12469587  0.1264389
  0.128107    0.12912719  0.1328247   0.13380619  0.12711915  0.1238633
  0.12191707  0.12051536  0.11946354  0.12045658  0.12086575  0.1237279
  0.12447031  0.12605418  0.12677349  0.12274314  0.12122535  0.12001892
  0.11877382  0.11859537  0.11958984  0.12140815  0.12192893  0.12393738
  0.12850611  0.12383677  0.12139611  0.12022088  0.1188489   0.1190173
  0.11788641  0.11867828  0.12113734  0.122239    0.13154297  0.12637977
  0.12379657  0.1212301   0.11940305  0.11920701  0.11899411  0.11917097
  0.11986611  0.1213416   0.13511688  0.13009119  0.127

In [17]:
import plotly.plotly as py
import plotly.graph_objs as go

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors_fast))]
c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]
data = []


x = np.repeat(l_rate_fast,new_trials)
y = np.tile(sp_rad_fast,new_trials)
z = fast_errorLists

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name=("Fast network averaged over",trials,"Trials"),
    mode='markers',
    marker=dict(
        size=7,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Fast Network Averaged')

In [18]:
data = [
     go.Contour(
        z=fast_errorLists.reshape(10,10),
        x=l_rate_fast,
        y=sp_rad_fast,
        colorscale='Jet',
    )]
py.iplot(data)